## hdf5 converter

source: http://machinelearninguru.com/deep_learning/data_preparation/hdf5/hdf5.html

In [1]:
from random import shuffle
import glob

shuffle_data = True  # shuffle the addresses before saving
hdf5_path = r'C:\Users\DevineauY\Documents\Audit\2019-Contract Management\2-Data sources\Signature\dataset.hdf5'  # address to save the hdf5 file
signature_train_path = r'C:\Users\DevineauY\Documents\Audit\2019-Contract Management\2-Data sources\Signature\*.png'

# read addresses and labels from the 'train' folder
addrs = glob.glob(signature_train_path)

labels = []
for addr in addrs:
    if 'foot' in addr: # signature footer
        labels.append(0)
    elif 'full' in addr: # signature main
        labels.append(1)
    elif 'FALSE' not in addr: # date handwritten
        labels.append(2)
    else:
        labels.append(3)

# to shuffle data
if shuffle_data:
    c = list(zip(addrs, labels))
    shuffle(c)
    addrs, labels = zip(*c)
    
# Divide the data into 60% train, 20% validation, and 20% test
train_addrs = addrs[0:int(0.9*len(addrs))]
train_labels = labels[0:int(0.9*len(labels))]

#val_addrs = addrs[int(0.6*len(addrs)):int(0.8*len(addrs))]
#val_labels = labels[int(0.6*len(addrs)):int(0.8*len(addrs))]

test_addrs = addrs[int(0.9*len(addrs)):]
test_labels = labels[int(0.9*len(labels)):]

### Dataset creation

In [2]:
import numpy as np
import h5py

w = 64
h = 64
data_order = 'tf'  # 'th' for Theano, 'tf' for Tensorflow

# check the order of data and chose proper data shape to save images
if data_order == 'th':
    train_shape = (len(train_addrs), 3, w, h)
    #val_shape = (len(val_addrs), 3, w, h)
    test_shape = (len(test_addrs), 3, w, h)
elif data_order == 'tf':
    train_shape = (len(train_addrs), w, h, 3)
    #val_shape = (len(val_addrs), w, h, 3)
    test_shape = (len(test_addrs), w, h, 3)
    
# open a hdf5 file and create earrays
hdf5_file = h5py.File(hdf5_path, mode='w')

hdf5_file.create_dataset("train_img", train_shape, np.uint8)
#hdf5_file.create_dataset("val_img", val_shape, np.int8)
hdf5_file.create_dataset("test_img", test_shape, np.uint8)

hdf5_file.create_dataset("train_mean", train_shape[1:], np.float32)

hdf5_file.create_dataset("train_labels", (len(train_addrs),), np.uint8)
hdf5_file["train_labels"][...] = train_labels

#hdf5_file.create_dataset("val_labels", (len(val_addrs),), np.int8)
#hdf5_file["val_labels"][...] = val_labels

hdf5_file.create_dataset("test_labels", (len(test_addrs),), np.uint8)
hdf5_file["test_labels"][...] = test_labels

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
try:
    from PIL import Image
except ImportError:
    import Image
    
def feed_dataset_h5(addrs,img_label, w, h):
    """
    Process the images and resize them to feed the data set 
    
    Arguments:
    dataset -- input data, of shape (input size, number of examples) (m, Hi, Wi, Ci)

    Returns:
     -- 
    """
    for i in range(len(addrs)):
    # print how many images are saved every 1000 images
        if i % 10 == 0 and i > 1:
            print('{} data: {}/{}'.format(img_label, i, len(addrs)))

        # read an image and resize to (w, h)
        # cv2 load images as BGR, convert it to RGB
        addr = addrs[i]
        img = Image.open(addr)
        img = img.resize((w,h))
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # add any image pre-processing here
        return img
        # save the image and calculate the mean so far
        #hdf5_file[img_label][i, ...] = img

In [4]:
import cv2
import matplotlib.pyplot as plt

def feed_dataset_h5(addrs,img_label, w, h):
    """
    Process the images and resize them to feed the data set 
    
    Arguments:
    dataset -- input data, of shape (input size, number of examples) (m, Hi, Wi, Ci)

    Returns:
     -- 
    """
    for i in range(len(addrs)):
    # print how many images are saved every 1000 images
        if i % 50 == 0 and i > 1:
            print('{} data: {}/{}'.format(img_label, i, len(addrs)))

        # read an image and resize to (w, h)
        # cv2 load images as BGR, convert it to RGB
        addr = addrs[i]
        img = cv2.imread(addr, cv2.IMREAD_UNCHANGED)
        img = cv2.resize(img, (w, h), interpolation=cv2.INTER_AREA)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # add any image pre-processing here
        #plt.imshow(img)
        # save the image and calculate the mean so far
        hdf5_file[img_label][i, ...] = img[None] 

In [5]:
feed_dataset_h5(train_addrs, "train_img", w, h)
#feed_dataset_h5(val_addrs, "val_img", w, h)
feed_dataset_h5(test_addrs, "test_img", w, h)
hdf5_file.close()

train_img data: 50/434
train_img data: 100/434
train_img data: 150/434
train_img data: 200/434
train_img data: 250/434
train_img data: 300/434
train_img data: 350/434
train_img data: 400/434


In [6]:
# open the hdf5 file
hdf5_file = h5py.File(hdf5_path, "r")

# Total number of samples
data_num = hdf5_file["train_img"].shape[0]
#val_num = hdf5_file["val_img"].shape[0]
test_num = hdf5_file["test_img"].shape[0]

In [7]:
data_num,test_num

(434, 49)

In [8]:
from random import shuffle
from math import ceil
import matplotlib.pyplot as plt

# create list of batches to shuffle the data
batches_list = list(range(int(ceil(float(data_num) / batch_size))))
shuffle(batches_list)

# loop over batches
for n, i in enumerate(batches_list):
    i_s = i * batch_size  # index of the first image in this batch
    i_e = min([(i + 1) * batch_size, data_num])  # index of the last image in this batch
    
    # read batch images and remove training mean
    images = hdf5_file.root.train_img[i_s:i_e]
    if subtract_mean:
        images -= mm
        
    # read labels and convert to one hot encoding
    labels = hdf5_file.root.train_labels[i_s:i_e]
    labels_one_hot = np.zeros((batch_size, nb_class))
    labels_one_hot[np.arange(batch_size), labels] = 1
    print(n+1, '/', len(batches_list))
    print(labels[0], labels_one_hot[0, :])
    plt.imshow(images[0])
    plt.show()
    
    if n == 5:  # break after 5 batches
        break
hdf5_file.close()

NameError: name 'batch_size' is not defined

In [ ]:
hdf5_file.close()